In [1]:
#!pip install git+https://github.com/modAL-python/modAL.git
#https://github.com/modAL-python/modAL

In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv('Breast_Cancer.csv', sep=',')
del df['Unnamed: 0']

df.head()

,ID,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,1,0,5,1,1,1,2,1,3,1,1
1,2,0,5,4,4,5,7,10,3,2,1
2,3,0,3,1,1,1,2,2,3,1,1
3,4,0,6,8,8,1,3,4,3,7,1
4,5,0,4,1,1,3,2,1,3,1,1


In [3]:
X=df.iloc[:,2:].values
y=df.iloc[:,1].values
X.shape, y.shape

((683, 9), (683,))

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train=X_train.reshape(-1,9)
y_train=y_train.reshape(-1,1)
X_test=X_test.reshape(-1,9)
y_test=y_test.reshape(-1,1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((457, 9), (457, 1), (226, 9), (226, 1))

## learner only trained on the training dataset ( i.e it did not learn greedily against the test dataset)

In [ ]:
from modAL.models import ActiveLearner
from modAL.uncertainty import entropy_sampling
from sklearn.ensemble import RandomForestClassifier
# initializing the learner
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=entropy_sampling,
    X_training=X_train, y_training=y_train
)


C:\Users\zecharpy\AppData\Local\Continuum\anaconda3\envs\car_plates_py36\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\zecharpy\AppData\Local\Continuum\anaconda3\envs\car_plates_py36\lib\site-packages\modAL\models\base.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(self.X_training, self.y_training, **fit_kwargs)


# learner query unseen dataset=X_test and find a query example to ask the oracle

In [ ]:
cols=df.iloc[:,2:].columns.tolist()
cols

['age',
 'menopause',
 'tumor-size',
 'inv-nodes',
 'node-caps',
 'deg-malig',
 'breast',
 'breast-quad',
 'irradiat']

In [ ]:
import seaborn as sns
sns.set(style="ticks", color_codes=True)
df_test=pd.DataFrame(X_test,columns=cols)
df_test['Class']=y_test
df_test.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,8,3,4,9,3,10,3,3,1,1
1,8,8,7,4,10,10,7,8,7,1
2,1,1,1,1,2,1,3,1,1,0
3,1,1,2,2,2,1,3,1,1,0
4,1,1,1,1,1,1,2,1,1,0


In [ ]:
print("pair-wised scatterplot with actual class")
g = sns.pairplot(df_test, hue="Class")

pair-wised scatterplot with actual class


# prediction made BEFORE active learning 

In [ ]:
y_pred_b4_AL=learner.predict(X_test)
y_pred_b4_AL.shape, X_test.shape

In [ ]:
df_test['pred_b4_AL']=y_pred_b4_AL
df_test['wrong_pred_b4'] = np.where((df_test['Class'] != df_test['pred_b4_AL']),1,0)
print("--------------------------- pair-wised scatterplot with predicted class BEFORE active learning -------------------")
g = sns.pairplot(df_test, hue="pred_b4_AL")

In [ ]:
how_many_queries=100
# query for labels
for i in range(how_many_queries):
    
    query_idx, query_inst = learner.query(X_test)
    learner.teach(X_test[query_idx], y_test[query_idx])
    out=learner.predict_proba(X_test[query_idx])
    print("predictive result after active learning :{}".format(str(np.argmax(out))))
    print("correct answer is :{}".format(str(y_test[query_idx][0][0])))

# prediction made AFTER active Learning 

In [ ]:
df_test.columns.tolist()[-3:] , len(df_test.columns)

In [ ]:
y_after=learner.predict(X_test)
df_test['pred_after_AL']=y_after


# check wrong prediction both before and after

In [ ]:
df_test['wrong_pred_af'] = np.where((df_test['Class'] != df_test['pred_after_AL']),1,0)
df_test.iloc[:,-5:].head(50)


In [ ]:
df_test.iloc[:,[-3,-1]].sum()